In [12]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import glob
import string
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from collections import Counter

In [13]:
os.getcwd()

'/home/yinghao/Documents/cancer blog/CancerBlog-Analytics/input data/individual blog data'

In [14]:
os.chdir('/home/yinghao/Documents/cancer blog/CancerBlog-Analytics/input data/individual blog data')

In [15]:
files = glob.glob('./*.txt')
file_list=[]
for names in files:
    f = open(names, 'rU')
    raw = f.read()
    file_list += [raw]

In [16]:
NLTK_STOPWORDS = set(nltk.corpus.stopwords.words('english'))
def remove_numbers_in_string(s):
    return s.translate(None, string.digits)

def lowercase_remove_punctuation(s):
    s = s.lower()
    s = s.translate(None, string.punctuation)
    return s
    
def remove_stopwords(s):
    token_list = nltk.word_tokenize(s)
    exclude_stopwords = lambda token : token not in NLTK_STOPWORDS
    return ' '.join(filter(exclude_stopwords, token_list))



def stem_token_list(token_list):
    STEMMER = PorterStemmer()
    return [STEMMER.stem(tok.decode('utf-8',errors='ignore')) for tok in token_list]

def restring_tokens(token_list):
    return ' '.join(token_list)

def lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem_and_restring(s):
    s = remove_numbers_in_string(s)
    s = lowercase_remove_punctuation(s)
    s = remove_stopwords(s)
    token_list = nltk.word_tokenize(s)
    token_list = stem_token_list(token_list)
    return restring_tokens(token_list)

In [21]:
 
file_cleaned =[]
%time
for i in file_list:
    k = lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem_and_restring(i)
    file_cleaned += [k]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.01 µs


In [7]:
len(file_cleaned)

1598

In [8]:
file_cleaned[1]

u'cant quit smoke may teeni tini bit ad protect possibl onset lung cancer will get sweatin studi older women research found physic activ smoker percent lower risk develop lung cancer sedentari counterpartsthi doesnt mean women exercis free pass smoke studi reveal lowest risk lung cancer among studi particip found moder workout four time per week vigor workout two time per weekit unclear physic activ might prevent effect lung cancer perhap improv pulmonari function result exercis reduc concentr carcinogen particl deposit lung mayb becom physic activ increas smoker awar damag caus lung motiv smoke less quiton thing clear howev singl import step smoker take prevent lung cancer quit nasti habit stop smoke time less like develop lung cancer smokedr norman edelman chief medic offic american lung associ want smoker gain fals sens secur studi dont want peopl get wrong messag edelman said regular smoker risk lung cancer time nonsmok percent reduct risk trivialedelman also state studi address ef

In [22]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, ngram_range=(1,3))
%time tfidf = tfidf_vectorizer.fit_transform(file_cleaned)

CPU times: user 4.64 s, sys: 140 ms, total: 4.78 s
Wall time: 4.64 s


In [10]:
tfidf.shape

(1598, 2577893)

In [23]:
km = KMeans(n_clusters=10)
%time km.fit(tfidf)

CPU times: user 14.3 s, sys: 2.09 s, total: 16.4 s
Wall time: 16.4 s


KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=10, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [24]:
order_centroids= km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()

In [25]:
for i in range(10):
    print 'Clusert %d' %i,
    for ind in order_centroids[i,:30]:
        print '%s;'%terms[ind],
    print

Clusert 0 red; blood; vitamin; blood cell; wine; cancer; glass; red blood cell; red blood; run; cell; bodi; level; spinach; good; one; exercis; eat; think; red wine; dont; feel; chemotherapi; port; bone; meat; oyster; low; need; take;
Clusert 1 cancer; time; life; one; day; year; know; hair; like; treatment; feel; love; would; get; breast; think; take; back; go; want; live; book; help; thing; friend; make; good; peopl; never; say;
Clusert 2 cell; drug; cancer; patient; tumor; treatment; research; use; test; studi; cancer cell; effect; trial; therapi; new; breast; chemotherapi; clinic; may; one; breast cancer; stem; target; stem cell; blood; diseas; treat; protein; work; develop;
Clusert 3 cancer; breast; women; breast cancer; risk; studi; mammogram; year; age; research; diagnos; increas; young; one; percent; screen; develop; may; detect; diseas; use; skin; say; test; earli; time; health; treatment; like; found;
Clusert 4 cancer; smoke; lung; lung cancer; children; eat; health; food; ri

In [26]:
clusters = km.labels_.tolist()
blogs = {'blog': file_cleaned, 'cluster': clusters}
frame = pd.DataFrame(blogs)

In [ ]:
frame.head()

In [17]:
frame['blog_original'] = file_list

In [18]:
frame.head()

,blog,cluster,blog_original
0,love oyster way everi way could everi day canc...,9,"I love oysters any way and every way, and if I..."
1,cant quit smoke may teeni tini bit ad protect ...,7,Those who just can't quit smoking may have a t...
2,way feel pain cancer experi without person end...,5,There is no way to feel the pain of a cancer e...
3,afflict cancer sometim life seem littl hard co...,4,"To those afflicted with cancer, sometimes life..."
4,new studi find chronic gum diseas could lead e...,3,A new study finds that chronic gum disease cou...


In [21]:
frame['blog_original'][4]

'A new study finds that chronic gum disease could lead to elevated future tongue cancer risks in men. So, instead of just brushing those teeth, be sure and massage and clean those gums. Oh, did I forget to mention flossing?Periodontitis -- aka, gum disease -- was looked at in this study insofar as studying the amount of bone loss in tooth cavities. Conclusions? Men with tongue cancer had significantly more bone loss in tooth cavities than those without tongue cancer.After the results were studied, the researchers believed that certain periodontal bacteria may be so toxic as to lead to tongue (oral) cancer. Moral of the story (for everyone) here: make sure that you treat that mouth to a good cleaning a few times a day with brushing and flossing. Comments: \n'

In [22]:
frame['cluster'].value_counts()

0    417
6    328
4    274
7    134
5    103
3     92
8     89
1     78
2     51
9     28
Name: cluster, dtype: int64

In [24]:
with open('allfiles.txt','a') as file_1:
    for i in file_cleaned:
        ii = i.encode('utf-8', errors='ignore')
        file_1.write(ii)

In [25]:
data = open('./allfiles.txt').read()

In [26]:
bigram = ngrams(data.split(), 2)
bigram_list = list(bigram)
bigram_list_count = Counter(bigram_list)
bigram_list_count.most_common()

[(('breast', 'cancer'), 3377),
 (('alt', 'hspace'), 598),
 (('alignright', 'vspace'), 588),
 (('cancer', 'patient'), 549),
 (('comment', 'img'), 539),
 (('prostat', 'cancer'), 406),
 (('cancer', 'survivor'), 387),
 (('cancer', 'cell'), 383),
 (('clinic', 'trial'), 356),
 (('side', 'effect'), 333),
 (('idvimag', 'alt'), 324),
 (('lung', 'cancer'), 297),
 (('img', 'idvimag'), 294),
 (('diagnos', 'breast'), 292),
 (('lymph', 'node'), 251),
 (('cancer', 'research'), 246),
 (('img', 'alt'), 244),
 (('cancer', 'treatment'), 243),
 (('ovarian', 'cancer'), 220),
 (('five', 'year'), 194),
 (('skin', 'cancer'), 183),
 (('year', 'ago'), 180),
 (('unit', 'state'), 178),
 (('year', 'old'), 177),
 (('young', 'women'), 176),
 (('cancer', 'societi'), 174),
 (('increas', 'risk'), 165),
 (('two', 'year'), 163),
 (('stem', 'cell'), 162),
 (('studi', 'show'), 154),
 (('immun', 'system'), 153),
 (('american', 'cancer'), 153),
 (('risk', 'develop'), 152),
 (('one', 'day'), 150),
 (('support', 'group'), 146)

In [27]:
trigram = ngrams(data.split(), 3)
trigram_list = list(trigram)
trigram_list_count = Counter(trigram_list)
trigram_list_count.most_common()

[(('idvimag', 'alt', 'hspace'), 324),
 (('comment', 'img', 'idvimag'), 292),
 (('img', 'idvimag', 'alt'), 290),
 (('comment', 'img', 'alt'), 244),
 (('img', 'alt', 'hspace'), 244),
 (('diagnos', 'breast', 'cancer'), 241),
 (('breast', 'cancer', 'survivor'), 202),
 (('american', 'cancer', 'societi'), 143),
 (('metastat', 'breast', 'cancer'), 112),
 (('breast', 'cancer', 'patient'), 95),
 (('nation', 'cancer', 'institut'), 88),
 (('risk', 'breast', 'cancer'), 86),
 (('breast', 'cancer', 'research'), 70),
 (('susan', 'g', 'komen'), 67),
 (('cell', 'cultur', 'assay'), 63),
 (('alignright', 'vspace', 'borderth'), 63),
 (('women', 'breast', 'cancer'), 61),
 (('cancer', 'breast', 'cancer'), 61),
 (('breast', 'cancer', 'treatment'), 60),
 (('breast', 'cancer', 'awar'), 59),
 (('develop', 'breast', 'cancer'), 56),
 (('circul', 'tumor', 'cell'), 53),
 (('advic', 'would', 'give'), 51),
 (('women', 'diagnos', 'breast'), 51),
 (('alignright', 'vspace', 'bordera'), 48),
 (('young', 'surviv', 'coalit

# 3D TEST

In [27]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
dist = 1 - cosine_similarity(tfidf)

In [21]:
mds = MDS(n_components=3, dissimilarity="precomputed", random_state=1)

pos = mds.fit_transform(dist)

In [41]:
fig = plt.figure(figsize=(16,12))
plt.clf()
ax = Axes3D(fig,rect=[0,0,.95,1], elev=48, azim = 134)
#ax.scatter(s=20)
plt.cla()
ax.scatter(pos[:,0], pos[:,1], pos[:,2], c=clusters, s = 20)
plt.show()

In [ ]:
plt.show()

In [ ]:
fig = plt.figure(figsize=(16,12))
plt.clf()
ax = Axes3D(fig,rect=[0,0,.95,1], elev=48, azim = 134)
#ax.scatter(s=20)
plt.cla()
ax.scatter(pos[:,0], pos[:,1], pos[:,2], c=clusters, s = 20)
plt.show()

In [29]:
mds_1 = MDS(n_components=2, dissimilarity="precomputed", random_state=1)

pos_1 = mds_1.fit_transform(dist)

In [30]:
pos_1[1]

array([-0.5732783 , -0.00366295])

In [34]:
plt.scatter(pos_1[:,0], pos_1[:,1], c=clusters)

In [35]:
plt.show()

In [36]:
test = pd.DataFrame(pos_1)

In [38]:
test['3'] = clusters

##### for i in range(10):
    a = test[test[3] == i]
    a[0] = a[0].apply(lambda x:(x-a.mean()[0])/a.std()[0])
    a[1] = a[1].apply(lambda x:(x-a.mean()[1])/a.std()[1])
    test = test.append(a)
    
    

In [61]:
new_data = test.iloc[1598:]

In [62]:
new_data.head()

,0,1,3
87,-0.672030,-1.294206,0
103,1.433728,-0.046206,0
136,0.529488,1.317239,0
257,-0.444666,-1.521771,0
362,-1.155589,0.395767,0


In [63]:
plt.scatter(new_data[0],new_data[1],c=new_data[3])
plt.show()

In [50]:
a = test[test[3] == 1]
a.describe()

,0,1,3
count,360.000000,360.000000,360
mean,-0.005169,-0.020633,1
std,0.474655,0.458592,0
min,-0.762166,-0.794469,1
25%,-0.444484,-0.452560,1
50%,0.001597,0.024328,1
75%,0.409007,0.385295,1
max,0.788688,0.779463,1


In [52]:
a = test[test[3] == 0]
a.std()

0    0.536414
1    0.448506
3    0.000000
dtype: float64

# Similarity Matrix

In [29]:
dist[0:9,0:9].tofile('./dist1.')

In [32]:
a = dist.tolist()

In [34]:
len(a[0])

1595

In [35]:
len(a)

1595

In [36]:
aa = np.array(a)

In [37]:
aa.shape

(1595, 1595)

In [40]:
np.savetxt('./dist2.txt',aa,delimiter=',')